In [1]:
import json
import pickle
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sensor import get_sensor_tuple_data

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
# from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier

In [2]:
with open("sensor_labels.pkl", "rb") as f:
    labels = pickle.load(f)

with open("interpolation_functions.pkl", "rb") as f:
    interp_funcs = pickle.load(f)

In [ ]:
random_state = 42
df_dict = {
    "Decision Tree": [],
    "GaussianNB": [],
    "BaggingTree": [],
    "SVM": [],
    "XGB": [],
    "Logistic Reg.": [],
    "MLP": [],
    "VotingClassifier": []
}

results = {"s_0_1": [], "s_2_3": [], "s_4_5": [], "s_6_7": []}


for s in range(0, 8, 2):
    s_l = s
    s_r = s+1
    print(f"Sensors {s_l} & {s_r}")

    X_0, y, _, _ = get_sensor_tuple_data(0, s_l, s_r, interp_funcs, labels)
    X_1, _, _, _ = get_sensor_tuple_data(1, s_l, s_r, interp_funcs, labels)
    X = X_0 - X_1

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    le = LabelEncoder()
    y_enc = le.fit_transform(y)

    classifiers = []

    # decision tree
    clf_name = "Decision Tree"
    dt = DecisionTreeClassifier(criterion="entropy",
                                random_state=random_state)
    accuracy = np.mean(cross_val_score(dt, X, y, cv=5))
    results[f"s_{s_l}_{s_r}"].append(
        {"clf": clf_name, "accuracy": np.round(accuracy, 3)}
    )
    df_dict[clf_name].append(np.round(accuracy, 3))
    # classifiers.append((clf_name, dt))

    # gaussian naive bayes
    clf_name = "GaussianNB"
    gnb = GaussianNB()
    accuracy = np.mean(cross_val_score(gnb, X, y, cv=5))
    results[f"s_{s_l}_{s_r}"].append(
        {"clf": clf_name, "accuracy": np.round(accuracy, 3)}
    )
    df_dict[clf_name].append(np.round(accuracy, 3))
    classifiers.append((clf_name, gnb))

    # bagging
    clf_name = "BaggingTree"
    extra_tree = ExtraTreeClassifier(random_state=random_state)
    bc = BaggingClassifier(extra_tree, random_state=random_state)
    accuracy = np.mean(cross_val_score(bc, X, y, cv=5))
    results[f"s_{s_l}_{s_r}"].append(
        {"clf": clf_name, "accuracy": np.round(accuracy, 3)}
    )
    df_dict[clf_name].append(np.round(accuracy, 3))
    classifiers.append((clf_name, bc))

    # svm
    clf_name = "SVM"
    svm = SVC(kernel="linear", random_state=random_state)
    accuracy = np.mean(cross_val_score(svm, X, y, cv=5))
    results[f"s_{s_l}_{s_r}"].append(
        {"clf": clf_name, "accuracy": np.round(accuracy, 3)}
    )
    df_dict[clf_name].append(np.round(accuracy, 3))
    # classifiers.append((clf_name, svm))

    # xgb
    clf_name = "XGB"
    xgb = XGBClassifier(objective="multi:softprob",
                        random_state=random_state)
    accuracy = np.mean(cross_val_score(xgb, X, y_enc, cv=5))
    results[f"s_{s_l}_{s_r}"].append(
        {"clf": "XGB", "accuracy": np.round(accuracy, 3)}
    )
    df_dict[clf_name].append(np.round(accuracy, 3))
    # classifiers.append((clf_name, xgb))

    # log. reg.
    clf_name = "Logistic Reg."
    lr = LogisticRegression(max_iter=5000, random_state=random_state)
    accuracy = np.mean(cross_val_score(lr, X, y, cv=5))
    results[f"s_{s_l}_{s_r}"].append(
        {"clf": clf_name, "accuracy": np.round(accuracy, 3)}
    )
    df_dict[clf_name].append(np.round(accuracy, 3))
    # classifiers.append((clf_name, lr))

    # mlp
    clf_name = "MLP"
    mlp = MLPClassifier(
        hidden_layer_sizes=(256, ),
        activation="relu",
        learning_rate="adaptive",
        max_iter=10000,
        random_state=random_state)
    accuracy = np.mean(cross_val_score(mlp, X, y, cv=5))
    results[f"s_{s_l}_{s_r}"].append(
        {"clf": clf_name, "accuracy": np.round(accuracy, 3)}
    )
    df_dict[clf_name].append(np.round(accuracy, 3))
    classifiers.append((clf_name, mlp))

    # voting clf
    clf_name = "VotingClassifier"
    vclf = VotingClassifier(estimators=classifiers, voting="soft")
    accuracy = np.mean(cross_val_score(vclf, X, y, cv=5))
    results[f"s_{s_l}_{s_r}"].append(
        {"clf": clf_name, "accuracy": np.round(accuracy, 3)}
    )
    df_dict[clf_name].append(np.round(accuracy, 3))

Sensors 0 & 1
Sensors 2 & 3
Sensors 4 & 5
Sensors 6 & 7


In [ ]:
import pandas as pd
df = pd.DataFrame(df_dict)
df.to_excel("model_accu_scores.xlsx")